In [ ]:
import os
from collections import Counter
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader,Dataset,random_split
import torchvision.transforms as T
from PIL import Image
import os
import pandas as pd
import torch
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from transformers import ViTImageProcessor, AutoTokenizer, VisionEncoderDecoderModel

In [ ]:
data_location =  "../input/flickr30k"
!ls $data_location

# in ra độ dài của file txt )gồm caption và hình ảnh tương ứng

caption_file = data_location + '/captions.txt'
df = pd.read_csv(caption_file)
print("There are {} image to captions".format(len(df)))
df.head(7)
df= df.astype(str)
print(df.info())

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# vẽ ra ảnh cùng 5 caption tương ứng của ảnh đó

data_idx = 0

image_path = data_location+"/Images/"+df.iloc[data_idx,0]
img=mpimg.imread(image_path)
plt.imshow(img)
plt.show()
image = Image.open(image_path)

for i in range(data_idx,data_idx+5):
    print("Caption:",df.iloc[i,1])
print(image.size)

In [ ]:
from transformers import VisionEncoderDecoderModel, AutoTokenizer, AutoFeatureExtractor
image_encoder_model = "google/vit-base-patch16-224-in21k"
text_decode_model = "gpt2"
device = ("cuda" if torch.cuda.is_available() else "cpu")
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(
    image_encoder_model, text_decode_model) # khởi tạo 1 model với cấu trúc gồm image encoder (vit-base-patch) và text decoder (gpt2)


feature_extractor = AutoFeatureExtractor.from_pretrained(image_encoder_model)
# feature extractor bản chất chỉ có tác dụng resize ảnh về 224x224 và rescale pixel values và [0,1]
# giúp để sẵn sàng đưa vào image encoder của model
tokenizer = AutoTokenizer.from_pretrained(text_decode_model)
tokenizer.pad_token = tokenizer.eos_token

#test the tokenizer
''' 
print(tokenizer.tokenize("i love vietnam"))
print(tokenizer.special_tokens_map)
print(tokenizer.convert_tokens_to_ids(tokenizer.special_tokens_map))
'''

In [ ]:
class FlickrDataset(Dataset): #khởi tạo dataset để đưa vào dataloader

    def __init__(self,root_dir,captions_file,tokenizer,transform=None,freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(caption_file)
        self.transform = transform
        
       
        self.imgs = self.df["image"]
        self.captions = self.df["caption"]
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,idx):
        caption = self.captions[idx] # lấy caption dựa trên index
        img_name = self.imgs[idx] # lấy tên ảnh dựa trên index 

        # tạo đường dẫn đến tệp ảnh dựa vào tên ảnh đã có và đổi về dạng RGB
        img_location = os.path.join(self.root_dir,img_name)
        img = Image.open(img_location).convert("RGB")
        if not isinstance(caption, str):
            caption = str(caption)
        if self.transform is not None:
            img = self.transform(img)    
        labels = tokenizer(caption, 
                      padding="max_length", 
                      max_length= 20,truncation= True).input_ids
        return img, torch.tensor(labels)

In [ ]:
transforms = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(), # chuyển về tensor và chuẩn hóa [0,1], thay tác dụng của feature extractor
])

In [ ]:
def show_image(inp, title=None):
    """Imshow for Tensor."""

    inp = inp.numpy().transpose((1, 2, 0))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

In [ ]:
data_location =  "../input/flickr30k"
valdata_location = "../input/flickr8k"
dataset =  FlickrDataset(
    root_dir = data_location+"/Images",
    captions_file = data_location+"/captions.txt",
    transform=transforms, tokenizer = tokenizer
)


In [ ]:
train_size = int(0.9*len(dataset))
valid_size = len(dataset) - train_size

In [ ]:
train_data,valid_data = random_split(dataset,[train_size,valid_size]) 
# chia train và valid thoe tỷ lệ 9:1

In [ ]:
print(len(dataset))
print(len(train_data))
print(len(valid_data))

In [ ]:
tokenizer.pad_token = tokenizer.eos_token # chuyển pad_token thành eos_token
img, caps= dataset[5]
show_image(img,"Image")
print("Token:",caps)
print(img.shape)
print(caps.shape)

In [ ]:
BATCH_SIZE = 32

In [ ]:
train_loader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=True,
)
valid_loader = DataLoader(dataset=valid_data,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKER,
    shuffle=False,
)

In [ ]:
print(len(train_loader))
for images,inputs_ids in train_loader:
    print(images.shape)
    print(inputs_ids.shape)
    image =images[0]
    inputs_id = inputs_ids[0]
    plt.imshow(image.permute(1,2,0))
    print("Input_id: ",inputs_id)
    break

In [ ]:
print(len(valid_loader))
for images,inputs_ids in valid_loader:
    print(images.shape)
    print(inputs_ids.shape)
    image =images[0]
    inputs_id = inputs_ids[0]
    plt.imshow(image.permute(1,2,0))
    print("Input_id: ",inputs_id)
    break

In [ ]:
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss

In [ ]:
lr = 1e-4
num_epoch = 10
optimizer = AdamW(model.parameters(),lr = lr)
loss = CrossEntropyLoss()
best_valid_loss = 10000000

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
model.config.eos_token_id = tokenizer.eos_token_id # chuyển kết thúc câu là eos token
model.config.decoder_start_token_id = tokenizer.bos_token_id #chuyển bắt đầu câu là token bos
model.config.pad_token_id = tokenizer.pad_token_id #chuyển token_pad là pad_token_id
tokenizer.pad_token = tokenizer.eos_token # tokenizer cũng pad bằng eos_token

In [ ]:
!pip install wandb
import wandb
!wandb login "1a1e9d904fb11812f635b8c3f9a93ae09da4cd04"

In [ ]:
wandb.init(project = "My final train model from 1412")

In [ ]:
from tqdm import tqdm
import numpy as np
for epoch in range(1, num_epoch + 1, 1):
    model.train() 
    print(f"Epoch: {epoch}")
    total_loss = 0

    progress_bar = tqdm(enumerate(train_loader, 1), total=len(train_loader), desc=f"Epoch {epoch}")

    for batch, (img, inputs_ids) in progress_bar:
        with torch.no_grad():
            img_encoded = feature_extractor(img, do_rescale=False)
        img_encoded = torch.tensor(np.array(img_encoded["pixel_values"])).to(device)
        inputs_ids = inputs_ids.to(device)


        output = model(labels=inputs_ids, pixel_values=img_encoded)
        loss = output.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

       
        progress_bar.set_postfix({"Batch Loss": loss.item()})
        if batch % 200 == 0:
            print(f"Batch {batch} of epoch{epoch} completed")
    avg_loss = total_loss / len(train_loader)
    wandb.log({"avg_train_loss": avg_loss, "epoch": epoch})
    print(f"Epoch {epoch}/{num_epoch}, Loss: {avg_loss:.4f}")
    
    model.eval() 
    total_val_loss = 0
    with torch.no_grad():  
        for batch, (img, inputs_ids) in tqdm(enumerate(valid_loader, 1), total=len(valid_loader), desc="Validation"):
            img_encoded = feature_extractor(img, do_rescale=False)
            img_encoded = torch.tensor(np.array(img_encoded["pixel_values"])).to(device)
            inputs_ids = inputs_ids.to(device)


            output = model(labels=inputs_ids, pixel_values=img_encoded)
            val_loss = output.loss
            total_val_loss += val_loss.item()
    
    avg_val_loss = total_val_loss / len(valid_loader)
    wandb.log({"avg_valid_loss": avg_val_loss, "epoch": epoch})
    print(f"Epoch {epoch}/{num_epoch}, Validation Loss: {avg_val_loss:.4f}")
    if avg_valid_loss < best_valid_loss:
        save_path = f"lastest_model.pth"
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_val_loss,
        }, save_path)
    print(f"Model saved to {save_path}")
